In [3]:
from pathlib import Path
from pydantic import BaseModel
from typing import Any
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os
import duckdb
load_dotenv()

True

In [4]:
class Neo4jGraph:

    def __init__(self, neo4j_uri:str, neo4j_username:str, neo4j_password:str, db:str)->None:
        self.uri  = neo4j_uri
        self.auth = (neo4j_username, neo4j_password)
        self.db = db
        self.driver = GraphDatabase.driver(self.uri, auth=self.auth)

    def query(self, query:str, params:dict):
        with self.driver.session(database=self.db) as session:
            result = session.run(query, params)
            return [r for r in result]
        
class Node(BaseModel):
    id:int
    label: list[str]
    properties:dict[str, Any]

class Relation(BaseModel):
    id:str
    label:str
    properties:dict[str, Any]


In [5]:
def merge_node(graph:Neo4jGraph, nodes:list[Node]):
    res = graph.query(
        "UNWIND $data as row "
        "CALL apoc.merge.node("
        "row.label, "
        "{id:row.id}, "
        "row.properties, "
        "row.properties ) "
        "YIELD node "
        "RETURN node"
        ,
        {
            "data":[
                node.__dict__ for node in nodes
            ]
        }
    )
    return res

In [6]:
base_path = Path().cwd().parent
source_path = base_path / Path('silver/anilist/character/character-2024-1-16.parquet')

neo4j_uri = os.environ['neo4j_uri']
neo4j_username = os.environ['neo4j_username']
neo4j_password = os.environ['neo4j_password']
neo4j_dbname = os.environ['neo4j_dbname']

In [7]:
graph = Neo4jGraph(
    neo4j_uri,
    neo4j_username,
    neo4j_password,
    neo4j_dbname,
)

In [10]:
tb = duckdb.read_parquet(str(source_path))
tb.shape

(10167, 20)

In [11]:
tb.order("character_id").limit(20).pl()

anime_id,character_id,role,role_name,character_id:1,name_first,name_middle,name_last,name_full,name_native,name_alternative,image,description,gender,dateOfBirth_year,dateOfBirth_month,dateOfBirth_day,age,bloodType,siteUrl
i32,i32,str,str,i32,str,str,str,str,str,list[str],str,str,str,i32,i32,i32,str,str,str
834,5,"""""MAIN""""",null,5,"""""Ichigo""""",null,"""""Kurosaki""""","""""Ichigo Kurosa…","""""黒崎一護""""","[""Ichi-nii"", ""Substitute Soul Reaper"", … ""Ryoka Boy""]","""""https://s4.an…","""""__Race:__ Hum…","""""Male""""",null,7,15,"""""15-29""""","""""A""""","""""https://anili…"
834,6,"""""MAIN""""",null,6,"""""Rukia""""",null,"""""Kuchiki""""","""""Rukia Kuchiki…","""""朽木ルキア""""",[],"""""https://s4.an…","""""__Race:__ Shi…","""""Female""""",null,1,14,null,null,"""""https://anili…"
664,11,"""""MAIN""""",null,11,"""""Edward""""",null,"""""Elric""""","""""Edward Elric""…","""""エドワード・エルリック""""","[""Ed"", ""Fullmetal Alchemist"", … ""Hero of the People""]","""""https://s4.an…","""""__Birthplace:…","""""Male""""",1899,null,null,"""""15-16 (series…",null,"""""https://anili…"
430,11,"""""MAIN""""",null,11,"""""Edward""""",null,"""""Elric""""","""""Edward Elric""…","""""エドワード・エルリック""""","[""Ed"", ""Fullmetal Alchemist"", … ""Hero of the People""]","""""https://s4.an…","""""__Birthplace:…","""""Male""""",1899,null,null,"""""15-16 (series…",null,"""""https://anili…"
430,12,"""""MAIN""""",null,12,"""""Alphonse""""",null,"""""Elric""""","""""Alphonse Elri…","""""アルフォンス・エルリック""…","[""Al"", ""Armored Alchemist""]","""""https://s4.an…",""""" __Birthplace…","""""Male""""",1900,null,null,"""""14-15""""",null,"""""https://anili…"
664,12,"""""MAIN""""",null,12,"""""Alphonse""""",null,"""""Elric""""","""""Alphonse Elri…","""""アルフォンス・エルリック""…","[""Al"", ""Armored Alchemist""]","""""https://s4.an…",""""" __Birthplace…","""""Male""""",1900,null,null,"""""14-15""""",null,"""""https://anili…"
1074,13,"""""MAIN""""",null,13,"""""Sasuke""""",null,"""""Uchiha""""","""""Sasuke Uchiha…","""""うちはサスケ""""","[""The Uchiha Survivor"", ""The Last Uchiha"", ""Sasuke of the Sharingan""]","""""https://s4.an…","""""__Height__: 1…","""""Male""""",null,7,23,"""""12-17(Naruto)…","""""AB""""","""""https://anili…"
594,13,"""""MAIN""""",null,13,"""""Sasuke""""",null,"""""Uchiha""""","""""Sasuke Uchiha…","""""うちはサスケ""""","[""The Uchiha Survivor"", ""The Last Uchiha"", ""Sasuke of the Sharingan""]","""""https://s4.an…","""""__Height__: 1…","""""Male""""",null,7,23,"""""12-17(Naruto)…","""""AB""""","""""https://anili…"
761,13,"""""MAIN""""",null,13,"""""Sasuke""""",null,"""""Uchiha""""","""""Sasuke Uchiha…","""""うちはサスケ""""","[""The Uchiha Survivor"", ""The Last Uchiha"", ""Sasuke of the Sharingan""]","""""https://s4.an…","""""__Height__: 1…","""""Male""""",null,7,23,"""""12-17(Naruto)…","""""AB""""","""""https://anili…"


In [34]:
doc_list = tb_anime.fetchmany(size=1000)
columns = tb_anime.columns

In [35]:
nodes = []
node_labels  = ['Anime']
for doc in doc_list:
    id_ = doc[0]
    prop = dict(zip(columns[1:], doc[1:]))
    node = Node(id=id_, label=node_labels, properties=prop)
    nodes.append(node)


In [60]:
res = merge_node(graph, nodes)